In [1]:
%pip install --force-reinstall -v "kagglehub==0.4.3"

Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
  Obtaining dependency information for kagglehub==0.4.3 from https://files.pythonhosted.org/packages/63/82/87c9ea391564debad2cf9828fe3afd7777145f3650a65fe124893d8f3008/kagglehub-0.4.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.7 MB/s eta 0:00:00
  Obtaining dependency information for kagglesdk<1.0,>=0.1.14 from https://files.pythonhosted.org/packages/1c/d8/71a8b9115ed2cf8b487c8976e66678311e82c73980e103eda15623abd6a5/kagglesdk-0.1.15-py3-none-any.whl.metadata
  Obtaining dependency information for packaging from https://files.pythonhosted.org/packages/b7/b9/c538f279a4e237a006a2c98387d081e9eb060d203d8ed34467cc0f0b9b53/packaging-26.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyyaml from https://files.pythonhosted.org/packages/8b/9d/b3589d3877982d4f2329302ef98a8026e7f4443c765c46cfecc8858c6b4b/pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylin

In [ ]:
import kagglehub
kagglehub.login()


In [ ]:
data = kagglehub.competition_download('shift-guard-10-robust-image-classification-challenge')
data

'/root/.cache/kagglehub/competitions/shift-guard-10-robust-image-classification-challenge'

In [16]:
import numpy as np 
import pandas as pd 

In [17]:
src_dir = "/root/.cache/kagglehub/competitions/shift-guard-10-robust-image-classification-challenge/"

In [14]:
train_labels = pd.read_csv(src_dir+'train_labels.csv')

train_labels.head(5)

,id,label
0,1,cat
1,2,dog
2,3,deer
3,4,automobile
4,5,bird


In [20]:
classes = pd.read_csv(src_dir+'classes.txt', sep=" ", header=None)
classes[0]

,0
0,airplane
1,automobile
2,bird
3,cat
4,deer
5,dog
6,frog
7,horse
8,ship
9,truck
